In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('EPL1819.csv') # read the dataset
df=df.drop_duplicates()
df.set_index(['MatchID'],inplace=True)
df1 = df.drop(['HLPos','ALPos','BkH','BkA','BkConf'],axis=1)

In [3]:
df = pd.read_csv('Test1920Data.csv') # read the dataset
df=df.drop_duplicates()
df.set_index(['MatchID'],inplace=True)
df1 = df.drop(['HLPos','ALPos','BkH','BkA','BkConf'],axis=1)

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 17 columns):
HTeam    287 non-null object
ATeam    287 non-null object
FTHG     287 non-null int64
FTAG     287 non-null int64
FTR      287 non-null object
HPo      287 non-null int64
APo      287 non-null int64
HS       287 non-null int64
AS       287 non-null int64
HF       287 non-null int64
AF       287 non-null int64
HC       287 non-null int64
AC       287 non-null int64
HYR      287 non-null int64
AYR      287 non-null int64
HPoss    287 non-null float64
APoss    287 non-null float64
dtypes: float64(2), int64(12), object(3)
memory usage: 40.4+ KB


In [5]:
df.head()

,HTeam,ATeam,HLPos,ALPos,FTHG,FTAG,FTR,HPo,APo,HS,...,AF,HC,AC,HYR,AYR,BkH,BkA,BkConf,HPoss,APoss
MatchID,,,,,,,,,,,,,,,,,,,,,
1,Liverpool,Norwich,Early,Early,4,1,H,3,0,7,...,9,11,2,0,2,Fav,Und,Confident,62.83,49.24
2,West Ham,Man City,Early,Early,0,5,A,0,3,3,...,13,1,1,2,2,Und,Fav,Confident,44.03,65.89
3,Bournemouth,Sheffield United,Early,Early,1,1,D,1,1,3,...,19,3,4,2,1,Fav,Und,Slightly_Confident,44.28,44.11
4,Burnley,Southampton,Early,Early,3,0,H,3,0,4,...,12,2,7,0,0,Fav,Und,Uncertain,41.55,47.59
5,Crystal Palace,Everton,Early,Early,0,0,D,1,1,2,...,14,6,2,2,2,Und,Fav,Slightly_Confident,44.31,49.66


In [6]:
Teams = df['HTeam'].drop_duplicates()
Teams.iloc[1]

'West Ham'

In [7]:
dd = {0: ['GS','FTHG','FTAG','AvgGS','HAvgGS','AAvgGS'], 1: ['GC','FTAG','FTHG','AvgGC','HAvgGC','AAvgGC'],
      2: ['PPG','HPo','APo','AvgPPG','HAvgPPG','AAvgPPG'], 3: ['F','HF','AF','AvgF','HAvgF','AAvgF'],
      4: ['ST','HS','AS','AvgST','HAvgST','AAvgST'], 5: ['SF','AS','HS','AvgSF','HAvgSF','AAvgSF'],
      6: ['C','HYR','AYR','AvgC','HAvgC','AAvgC'], 7: ['Co','HC','AC','AvgCo','HAvgCo','AAvgCo'], 
      8: ['Po','HPoss','APoss','AvgPo','HAvgPo','AAvgPo']}
dd

{0: ['GS', 'FTHG', 'FTAG', 'AvgGS', 'HAvgGS', 'AAvgGS'],
 1: ['GC', 'FTAG', 'FTHG', 'AvgGC', 'HAvgGC', 'AAvgGC'],
 2: ['PPG', 'HPo', 'APo', 'AvgPPG', 'HAvgPPG', 'AAvgPPG'],
 3: ['F', 'HF', 'AF', 'AvgF', 'HAvgF', 'AAvgF'],
 4: ['ST', 'HS', 'AS', 'AvgST', 'HAvgST', 'AAvgST'],
 5: ['SF', 'AS', 'HS', 'AvgSF', 'HAvgSF', 'AAvgSF'],
 6: ['C', 'HYR', 'AYR', 'AvgC', 'HAvgC', 'AAvgC'],
 7: ['Co', 'HC', 'AC', 'AvgCo', 'HAvgCo', 'AAvgCo'],
 8: ['Po', 'HPoss', 'APoss', 'AvgPo', 'HAvgPo', 'AAvgPo']}

In [8]:
df['HTPR'] = np.nan
df['ATPR'] = np.nan

In [ ]:
for j in range(len(dd)):
    df[dd[j][-2]] = np.nan
    df[dd[j][-1]] = np.nan
    for i in range(len(Teams)):
        a1 = df1[(df1['HTeam'] == Teams.iloc[i])|(df1['ATeam'] == Teams.iloc[i])]
        a1['MNo'] = range(0, len(a1))
        a1[dd[j][0]] = a1.apply(lambda row: row[dd[j][1]] if (row['HTeam'] == Teams.iloc[i]) else row[dd[j][2]], axis=1)
        a1[dd[j][0]] = a1[dd[j][0]].shift(1)
        a1[dd[j][3]] = a1[dd[j][0]].expanding().mean()
        a2 = a1[a1['HTeam'] == Teams.iloc[i]][[dd[j][3]]]
        a2.rename(columns={dd[j][3]:dd[j][4]},inplace=True)
        a3 = a1[a1['ATeam'] == Teams.iloc[i]][[dd[j][3]]]
        a3.rename(columns={dd[j][3]:dd[j][5]},inplace=True)
        a4 = df1[(df1['ATeam'] == Teams.iloc[i])]
        a4['ATPR'] = a4['FTR'].shift(1)
        a5 = df1[(df1['HTeam'] == Teams.iloc[i])]
        a5['HTPR'] = a5['FTR'].shift(1)
        df.loc[df.index.isin(a2.index),[dd[j][4]]] = a2[[dd[j][4]]]
        df.loc[df.index.isin(a3.index),[dd[j][5]]] = a3[[dd[j][5]]]
        df.loc[df.index.isin(a4.index),'ATPR'] = a4['ATPR']
        df.loc[df.index.isin(a5.index),'HTPR'] = a5['HTPR']

In [10]:
df2 = df.drop(['FTHG','FTAG','HPo','APo','HS','AS','HF','AF','HC','AC','HYR','AYR','HPoss','APoss'],axis=1)
df2.head(30)

,HTeam,ATeam,HLPos,ALPos,FTR,BkH,BkA,BkConf,HTPR,ATPR,...,HAvgST,AAvgST,HAvgSF,AAvgSF,HAvgC,AAvgC,HAvgCo,AAvgCo,HAvgPo,AAvgPo
MatchID,,,,,,,,,,,,,,,,,,,,,
1,Liverpool,Norwich,Early,Early,H,Fav,Und,Confident,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,West Ham,Man City,Early,Early,A,Und,Fav,Confident,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bournemouth,Sheffield United,Early,Early,D,Fav,Und,Slightly_Confident,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Burnley,Southampton,Early,Early,H,Fav,Und,Uncertain,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Crystal Palace,Everton,Early,Early,D,Und,Fav,Slightly_Confident,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Watford,Brighton,Early,Early,A,Fav,Und,Slightly_Confident,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Tottenham,Aston Villa,Early,Early,H,Fav,Und,Confident,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Leicester,Wolves,Early,Early,D,Fav,Und,Slightly_Confident,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Newcastle,Arsenal,Early,Early,A,Und,Fav,Confident,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287 entries, 1 to 287
Data columns (total 28 columns):
HTeam      287 non-null object
ATeam      287 non-null object
HLPos      287 non-null object
ALPos      287 non-null object
FTR        287 non-null object
BkH        287 non-null object
BkA        287 non-null object
BkConf     287 non-null object
HTPR       267 non-null object
ATPR       267 non-null object
HAvgGS     277 non-null float64
AAvgGS     277 non-null float64
HAvgGC     277 non-null float64
AAvgGC     277 non-null float64
HAvgPPG    277 non-null float64
AAvgPPG    277 non-null float64
HAvgF      277 non-null float64
AAvgF      277 non-null float64
HAvgST     277 non-null float64
AAvgST     277 non-null float64
HAvgSF     277 non-null float64
AAvgSF     277 non-null float64
HAvgC      277 non-null float64
AAvgC      277 non-null float64
HAvgCo     277 non-null float64
AAvgCo     277 non-null float64
HAvgPo     277 non-null float64
AAvgPo     277 non-null float64
dtypes: f

In [34]:
df2.to_excel (r'C:\Users\Kashmeel Bisseru\Downloads\EPL1819FinalP.xlsx', index = None, header=True)

In [14]:
df2.to_excel (r'EPL1920Test.xlsx', index = None, header=True)

In [14]:
a2 = df1[(df1['ATeam'] == Teams.iloc[0])]
a2['ATPR'] = a2['FTR'].shift(1)
a2

C:\Program1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,HTeam,ATeam,FTHG,FTAG,FTR,HPo,APo,HS,AS,HF,AF,HC,AC,HYR,AYR,HPoss,APoss,ATPR
MatchID,,,,,,,,,,,,,,,,,,
17,Brighton,Man United,3,2,H,3,0,3,3,16,13,3,5,1,1,40,60,NaN
38,Burnley,Man United,0,2,A,0,3,2,9,7,13,2,5,2,4,45,55,H
47,Watford,Man United,1,2,A,0,3,5,6,9,11,6,8,2,2,46,54,A
67,West Ham,Man United,3,1,H,3,0,3,4,12,12,4,9,0,1,49,51,A
83,Chelsea,Man United,2,2,D,1,1,6,4,9,17,5,3,2,5,59,41,H
102,Bournemouth,Man United,1,2,A,0,3,7,8,12,12,9,5,4,3,49,51,D
120,Man City,Man United,3,1,H,3,0,5,1,12,12,5,1,1,1,63,37,A
137,Southampton,Man United,2,2,D,1,1,6,5,12,13,3,5,4,4,44,56,H
169,Liverpool,Man United,3,1,H,3,0,11,2,6,14,13,2,0,2,64,36,D


In [13]:
a1 = df1[(df1['HTeam'] == Teams.iloc[0])]
a1['HTPR'] = a1['FTR'].shift(1)
a1

C:\Program1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,HTeam,ATeam,FTHG,FTAG,FTR,HPo,APo,HS,AS,HF,AF,HC,AC,HYR,AYR,HPoss,APoss,HTPR
MatchID,,,,,,,,,,,,,,,,,,
1,Man United,Leicester,2,1,H,3,0,6,4,11,8,2,5,2,1,44,56,NaN
30,Man United,Tottenham,0,3,A,0,3,5,5,11,16,5,2,2,4,52,48,H
58,Man United,Wolves,1,1,D,1,1,6,8,5,17,5,4,1,1,63,37,A
75,Man United,Newcastle,3,2,H,3,0,10,8,16,8,10,6,2,2,66,34,D
99,Man United,Everton,2,1,H,3,0,10,6,15,12,8,4,2,1,52,48,H
124,Man United,Crystal Palace,0,0,D,1,1,5,2,13,12,10,3,1,2,58,42,H
148,Man United,Arsenal,2,2,D,1,1,7,4,13,10,4,4,3,3,48,52,D
157,Man United,Fulham,4,1,H,3,0,11,4,11,15,10,3,1,2,59,41,D
187,Man United,Huddersfield,3,1,H,3,0,10,2,9,13,5,3,1,1,63,37,H


In [9]:
a1 = df1[(df1['HTeam'] == Teams.iloc[0])|(df1['ATeam'] == Teams.iloc[0])]
a1['MNo'] = range(0, len(a1))
a1['GS'] = a1.apply(lambda row: row['FTHG'] if (row['HTeam'] == Teams.iloc[0]) else row['FTAG'], axis=1)
a1['GS'] = a1['GS'].shift(1)
a1

C:\Program1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,HTeam,ATeam,FTHG,FTAG,FTR,HPo,APo,HS,AS,HF,AF,HC,AC,HYR,AYR,HPoss,APoss,MNo
MatchID,,,,,,,,,,,,,,,,,,
1,Man United,Leicester,2,1,H,3,0,6,4,11,8,2,5,2,1,44,56,0
17,Brighton,Man United,3,2,H,3,0,3,3,16,13,3,5,1,1,40,60,1
30,Man United,Tottenham,0,3,A,0,3,5,5,11,16,5,2,2,4,52,48,2
38,Burnley,Man United,0,2,A,0,3,2,9,7,13,2,5,2,4,45,55,3
47,Watford,Man United,1,2,A,0,3,5,6,9,11,6,8,2,2,46,54,4
58,Man United,Wolves,1,1,D,1,1,6,8,5,17,5,4,1,1,63,37,5
67,West Ham,Man United,3,1,H,3,0,3,4,12,12,4,9,0,1,49,51,6
75,Man United,Newcastle,3,2,H,3,0,10,8,16,8,10,6,2,2,66,34,7
83,Chelsea,Man United,2,2,D,1,1,6,4,9,17,5,3,2,5,59,41,8


In [50]:
#a1['CumGS'] = a1['GS'].cumsum()
a1['AvgGS'] = a1['GS'].expanding().mean()
a1['AvgGSP5'] = a1['GS'].rolling(window=5).mean()
a1.head(10)

C:\Program1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Program1\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,HTeam,ATeam,FTHG,FTAG,HPo,APo,HS,AS,HF,AF,HC,AC,HYR,AYR,HPoss,APoss,MNo,GS,AvgGS,AvgGSP5
MatchID,,,,,,,,,,,,,,,,,,,,
1,Man United,Leicester,2,1,3,0,8,13,11,8,2,5,2,1,44,56,0,NaN,NaN,NaN
17,Brighton,Man United,3,2,3,0,6,9,16,13,3,5,1,1,40,60,1,2.0,2.000000,NaN
30,Man United,Tottenham,0,3,0,3,23,9,11,16,5,2,2,4,52,48,2,2.0,2.000000,NaN
38,Burnley,Man United,0,2,0,3,9,21,7,13,2,5,2,4,45,55,3,0.0,1.333333,NaN
47,Watford,Man United,1,2,0,3,14,9,9,11,6,8,2,2,46,54,4,2.0,1.500000,NaN
58,Man United,Wolves,1,1,1,1,15,11,5,17,5,4,1,1,63,37,5,2.0,1.600000,1.6
67,West Ham,Man United,3,1,3,0,8,9,12,12,4,9,0,1,49,51,6,1.0,1.500000,1.4
75,Man United,Newcastle,3,2,3,0,18,13,16,8,10,6,2,2,66,34,7,1.0,1.428571,1.2
83,Chelsea,Man United,2,2,1,1,21,7,9,17,5,3,2,5,59,41,8,3.0,1.625000,1.8


In [61]:
a2 = a1[a1['HTeam'] == Teams.iloc[0]][['AvgGS','AvgGSP5']]
a2.rename(columns={'AvgGS':'HAvgGS','AvgGSP5':'HAvgGSP5'},inplace=True)
a3 = a1[a1['ATeam'] == Teams.iloc[0]][['AvgGS','AvgGSP5']]
a3.rename(columns={'AvgGS':'AAvgGS','AvgGSP5':'AAvgGSP5'},inplace=True)

In [79]:
#df = pd.merge(df,a2,on='MatchID',how='left')
df['HAvgGS'] = np.nan
df['HAvgGSP5'] = np.nan
df['AAvgGS'] = np.nan
df['AAvgGSP5'] = np.nan
df.head()
df.loc[df.index.isin(a2.index),['HAvgGS','HAvgGSP5']] = a2[['HAvgGS','HAvgGSP5']]
df.loc[df.index.isin(a3.index),['AAvgGS','AAvgGSP5']] = a3[['AAvgGS','AAvgGSP5']]
df

,HTeam,ATeam,HLPos,ALPos,FTHG,FTAG,FTR,HPo,APo,HS,...,AYR,BkH,BkA,BkConf,HPoss,APoss,HAvgGS,HAvgGSP5,AAvgGS,AAvgGSP5
MatchID,,,,,,,,,,,,,,,,,,,,,
1,Man United,Leicester,Early,Early,2,1,H,3,0,8,...,1,Clear Fav,Clear Und,Moderately Confident,44,56,NaN,NaN,NaN,NaN
2,Bournemouth,Cardiff,Early,Early,2,0,H,3,0,12,...,1,Fav,Und,Moderately Confident,62,38,NaN,NaN,NaN,NaN
3,Fulham,Crystal Palace,Early,Early,0,2,A,0,3,15,...,2,Slight Fav,Slight Und,Uncertain,60,40,NaN,NaN,NaN,NaN
4,Huddersfield,Chelsea,Early,Early,0,3,A,0,3,6,...,1,Clear Und,Clear Fav,Moderately Confident,40,60,NaN,NaN,NaN,NaN
5,Newcastle,Tottenham,Early,Early,1,2,A,0,3,15,...,2,Und,Fav,Slightly Confident,44,56,NaN,NaN,NaN,NaN
6,Watford,Brighton,Early,Early,2,0,H,3,0,19,...,2,Fav,Und,Slightly Confident,56,44,NaN,NaN,NaN,NaN
7,Wolves,Everton,Early,Early,2,2,D,1,1,11,...,2,Slight Fav,Slight Und,Slightly Confident,57,43,NaN,NaN,NaN,NaN
8,Arsenal,Man City,Early,Early,0,2,A,0,3,9,...,2,Und,Fav,Moderately Confident,40,60,NaN,NaN,NaN,NaN
9,Liverpool,West Ham,Early,Early,4,0,H,3,0,18,...,2,Clear Fav,Clear Und,Confident,61,39,NaN,NaN,NaN,NaN


In [82]:
a1 = df1[(df1['HTeam'] == Teams.iloc[1])|(df1['ATeam'] == Teams.iloc[1])]
a1['MNo'] = range(0, len(a1))
a1['GS'] = a1.apply(lambda row: row['FTHG'] if (row['HTeam'] == Teams.iloc[1]) else row['FTAG'], axis=1)
a1['GS'] = a1['GS'].shift(1)
a1['AvgGS'] = a1['GS'].expanding().mean()
a1['AvgGSP5'] = a1['GS'].rolling(window=5).mean()
a2 = a1[a1['HTeam'] == Teams.iloc[1]][['AvgGS','AvgGSP5']]
a2.rename(columns={'AvgGS':'HAvgGS','AvgGSP5':'HAvgGSP5'},inplace=True)
a3 = a1[a1['ATeam'] == Teams.iloc[1]][['AvgGS','AvgGSP5']]
a3.rename(columns={'AvgGS':'AAvgGS','AvgGSP5':'AAvgGSP5'},inplace=True)
df.loc[df.index.isin(a2.index),['HAvgGS','HAvgGSP5']] = a2[['HAvgGS','HAvgGSP5']]
df.loc[df.index.isin(a3.index),['AAvgGS','AAvgGSP5']] = a3[['AAvgGS','AAvgGSP5']]
df

C:\Program1\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Program1\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Program1\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.

,HTeam,ATeam,HLPos,ALPos,FTHG,FTAG,FTR,HPo,APo,HS,...,AYR,BkH,BkA,BkConf,HPoss,APoss,HAvgGS,HAvgGSP5,AAvgGS,AAvgGSP5
MatchID,,,,,,,,,,,,,,,,,,,,,
1,Man United,Leicester,Early,Early,2,1,H,3,0,8,...,1,Clear Fav,Clear Und,Moderately Confident,44,56,NaN,NaN,NaN,NaN
2,Bournemouth,Cardiff,Early,Early,2,0,H,3,0,12,...,1,Fav,Und,Moderately Confident,62,38,NaN,NaN,NaN,NaN
3,Fulham,Crystal Palace,Early,Early,0,2,A,0,3,15,...,2,Slight Fav,Slight Und,Uncertain,60,40,NaN,NaN,NaN,NaN
4,Huddersfield,Chelsea,Early,Early,0,3,A,0,3,6,...,1,Clear Und,Clear Fav,Moderately Confident,40,60,NaN,NaN,NaN,NaN
5,Newcastle,Tottenham,Early,Early,1,2,A,0,3,15,...,2,Und,Fav,Slightly Confident,44,56,NaN,NaN,NaN,NaN
6,Watford,Brighton,Early,Early,2,0,H,3,0,19,...,2,Fav,Und,Slightly Confident,56,44,NaN,NaN,NaN,NaN
7,Wolves,Everton,Early,Early,2,2,D,1,1,11,...,2,Slight Fav,Slight Und,Slightly Confident,57,43,NaN,NaN,NaN,NaN
8,Arsenal,Man City,Early,Early,0,2,A,0,3,9,...,2,Und,Fav,Moderately Confident,40,60,NaN,NaN,NaN,NaN
9,Liverpool,West Ham,Early,Early,4,0,H,3,0,18,...,2,Clear Fav,Clear Und,Confident,61,39,NaN,NaN,NaN,NaN


# PORTO WEATHER DATA

In [98]:
fd = pd.read_csv('PortoWeather2016.csv')
fd.head()

,Avg_TemperatureC,AvgT,Avg_DewpointC,Avg_PressurehPa,Avg_WindSpeedKMH,Avg_WindSpeedGustKMH,Avg_Humidity,Avg_HourlyPrecipMM,AvgHPre,Avg_dailyrainMM,...,Min_Humidity,StdDev_TemperatureC,StdDev_DewpointC,StdDev_PressurehPa,StdDev_WindSpeedKMH,StdDev_WindSpeedGustKMH,StdDev_Humidity,StdDev_HourlyPrecipMM,StdDev_dailyrainMM,StdDev_SolarRadiationWatts_m2
0,12.028125,10to15,10.111189,1016.938112,8.220629,21.795804,83.482517,0.713986,Heavy,8.171329,...,71,1.194010,1.719939,2.493384,6.771946,9.942910,6.925015,2.119371,7.081457,44.846179
1,13.751748,10to15,8.965278,1021.622222,3.406250,10.944444,81.600694,0.030903,Light,1.067708,...,70,1.110705,0.793380,1.111043,3.902473,5.867738,5.184095,0.357961,0.382681,133.540934
2,12.161053,10to15,12.759790,1013.204545,8.885664,22.276573,93.594406,2.858392,Heavy,26.113986,...,81,0.964591,1.752039,2.597005,6.734736,10.933841,5.936196,2.996697,24.665150,7.870713
3,8.889324,5to10,11.561754,1006.712982,0.388772,2.678596,95.971930,2.590877,Heavy,38.488772,...,71,1.441070,1.944741,2.655278,1.947917,4.543429,5.215691,2.982246,22.679284,15.742470
4,11.074912,10to15,6.711032,1012.766192,2.897153,11.038434,85.989324,1.242349,Heavy,19.327046,...,74,0.826023,0.514489,2.392659,6.077887,12.848503,4.318965,3.858250,9.126914,74.383235


In [99]:
fd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 34 columns):
Avg_TemperatureC                 355 non-null float64
AvgT                             355 non-null object
Avg_DewpointC                    355 non-null float64
Avg_PressurehPa                  355 non-null float64
Avg_WindSpeedKMH                 355 non-null float64
Avg_WindSpeedGustKMH             355 non-null float64
Avg_Humidity                     355 non-null float64
Avg_HourlyPrecipMM               355 non-null float64
AvgHPre                          355 non-null object
Avg_dailyrainMM                  355 non-null float64
AvgDRain                         355 non-null object
Avg_SolarRadiationWatts_m2       355 non-null float64
Max_TemperatureC                 355 non-null float64
Max_DewpointC                    355 non-null float64
Max_PressurehPa                  355 non-null float64
Max_WindSpeedKMH                 355 non-null float64
Max_WindSpeedGustKMH             3

In [100]:
fd1 = fd.drop(['Avg_TemperatureC','AvgT','Avg_HourlyPrecipMM','Avg_dailyrainMM', 'AvgHPre','AvgDRain'],axis=1)
fd1.head()

,Avg_DewpointC,Avg_PressurehPa,Avg_WindSpeedKMH,Avg_WindSpeedGustKMH,Avg_Humidity,Avg_SolarRadiationWatts_m2,Max_TemperatureC,Max_DewpointC,Max_PressurehPa,Max_WindSpeedKMH,...,Min_Humidity,StdDev_TemperatureC,StdDev_DewpointC,StdDev_PressurehPa,StdDev_WindSpeedKMH,StdDev_WindSpeedGustKMH,StdDev_Humidity,StdDev_HourlyPrecipMM,StdDev_dailyrainMM,StdDev_SolarRadiationWatts_m2
0,10.111189,1016.938112,8.220629,21.795804,83.482517,20.797203,15.1,13.9,1021.9,30.6,...,71,1.194010,1.719939,2.493384,6.771946,9.942910,6.925015,2.119371,7.081457,44.846179
1,8.965278,1021.622222,3.406250,10.944444,81.600694,77.868056,14.2,10.0,1023.6,16.1,...,70,1.110705,0.793380,1.111043,3.902473,5.867738,5.184095,0.357961,0.382681,133.540934
2,12.759790,1013.204545,8.885664,22.276573,93.594406,4.531469,15.4,15.0,1019.2,30.6,...,81,0.964591,1.752039,2.597005,6.734736,10.933841,5.936196,2.996697,24.665150,7.870713
3,11.561754,1006.712982,0.388772,2.678596,95.971930,9.287719,15.2,15.0,1011.7,20.9,...,71,1.441070,1.944741,2.655278,1.947917,4.543429,5.215691,2.982246,22.679284,15.742470
4,6.711032,1012.766192,2.897153,11.038434,85.989324,33.185053,10.8,7.8,1017.5,45.1,...,74,0.826023,0.514489,2.392659,6.077887,12.848503,4.318965,3.858250,9.126914,74.383235


In [101]:
gg = pd.DataFrame()

In [102]:
for i in fd1.columns[0:5]:
    gg[i] = pd.qcut(fd1[i], q=4)

In [105]:
fd2 = fd[['AvgT','AvgHPre','AvgDRain']].join(gg)

In [106]:
fd2.to_csv(r'C:\Users\Kashmeel Bisseru\Downloads\PW2016.csv', index = None, header=True)